In [1]:
import pandas as pd
import numpy as np
import feather
#import statsmodels.formula.api as smf
#from statsmodels.iolib.summary2 import summary_col
#import seaborn as sns
#import matplotlib.pyplot as plt
import os
#import altair as alt
from pathlib import Path

## Import data 

In [2]:
datadir = "../input"
#figuredir = "/mnt/PATSTAT/Solar/out/figures"
#Path(figuredir).mkdir(parents=True, exist_ok=True) # Create if doesn't exist

In [3]:
os.chdir(datadir)
patents_df = pd.read_feather("solar_dataset_all_renewables") 
patents_df.head()

,appln_id,appln_auth,appln_kind,appln_filing_date,appln_filing_year,receiving_office,earliest_filing_year,earliest_filing_id,earliest_publn_year,granted,...,nb_citing_docdb_fam,ipc_class_symbol,ipc_class_level,ipc_value,fossil_fuel,person_ctry_code,cpc_class_symbol,cpc_gener_auth,cpc_position,cpc_value
0,66,EP,A,2004-03-02,2004,,2003,905739597,2005,Y,...,14,F27B5/04,A,I,1.0,FR,None,None,None,None
1,103,EP,A,2006-04-13,2006,,2005,6502820,2007,N,...,4,F01K21/04,A,I,1.0,CH,None,None,None,None
2,127,EP,A,2006-10-02,2006,,2005,1109097,2008,Y,...,10,F27D3/02,A,I,1.0,AT,None,None,None,None
3,148,EP,A,2007-09-27,2007,,2006,15054845,2008,Y,...,11,F27B17/02,A,I,1.0,AT,None,None,None,None
4,256,EP,A,2008-06-06,2008,,2007,55458188,2009,Y,...,5,F23N1/02,A,I,1.0,DE,None,None,None,None


In [4]:
patents_df["country_code"] = patents_df['person_ctry_code']
# patents_df["country_code"] = patents_df["appln_auth"] # cover all observations but assign the authority country not the innovator country. This mostly concerns Chinese authority applications. 

In [5]:
# Classify patents as fossil fuel or renewables

patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 0

# Solar only:
patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 0

## Count patents 

In [6]:
### Absolute count ###

def absolute_count(df):
    global North

    ## fossil fuel ##
    solar_year = df[df.fossil_fuel == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique().to_frame(name = "Nb_ff_energy_innov_per_year") # Transform the series that groupby creates to a dataframe
    solar_year["Nb_ff_energy_app_per_year"] = df[df.fossil_fuel == 1].groupby('earliest_filing_year')['appln_id'].nunique()

    # countries #
    dict = {} 
    for country in list_country:
        name = "Nb_ff_energy_innov_per_year_" + country
        value = df[(df.fossil_fuel == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique() 
        dict["Nb_ff_energy_innov_per_year_" + country] = value
        solar_year[name] = dict[name] 

    # "north" countries #
    column_ff_names_North = []
    North = [x for x in list_country if x != 'CN']
    for country in North :
        name = "Nb_ff_energy_innov_per_year_" + country
        column_ff_names_North.append(name)
    solar_year['Nb_ff_energy_innov_per_year_North']= solar_year[column_ff_names_North].sum(axis=1)


    ## Solar panel ##
    solar_year["Nb_solar_energy_innov_per_year"] = df[df["cpc_renewables"] == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique()
    solar_year["Nb_solar_energy_app_per_year"] = df[df["cpc_renewables"] == 1].groupby('earliest_filing_year')['appln_id'].nunique()

    # countries #
    # by family
    dict = {} 
    for country in list_country:
        name = "Nb_solar_energy_innov_per_year_" + country
        value = df[(df["cpc_renewables"] == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique()
        dict["Nb_solar_energy_innov_per_year_" + country] = value
        solar_year[name] = dict[name] 

    # by application
    dict1 = {} 
    for country in list_country:
        name = "Nb_solar_energy_app_per_year_" + country
        value = df[(df["cpc_renewables"] == 1) & (df.country_code == country)].groupby('earliest_filing_year')['appln_id'].nunique()
        dict1["Nb_solar_energy_app_per_year_" + country] = value
        solar_year[name] = dict1[name] 

    # "north" countries #
    column_solar_names_North = []
    North = [x for x in list_country if x != 'CN']
    for country in North :
        name = "Nb_solar_energy_innov_per_year_" + country
        column_solar_names_North.append(name)
    solar_year['Nb_solar_energy_innov_per_year_North']= solar_year[column_solar_names_North].sum(axis=1)

    
    return solar_year

In [7]:
list_country = ["CN", "FR"]
absolute_count(patents_df)

,Nb_ff_energy_innov_per_year,Nb_ff_energy_app_per_year,Nb_ff_energy_innov_per_year_CN,Nb_ff_energy_innov_per_year_FR,Nb_ff_energy_innov_per_year_North,Nb_solar_energy_innov_per_year,Nb_solar_energy_app_per_year,Nb_solar_energy_innov_per_year_CN,Nb_solar_energy_innov_per_year_FR,Nb_solar_energy_app_per_year_CN,Nb_solar_energy_app_per_year_FR,Nb_solar_energy_innov_per_year_North
earliest_filing_year,,,,,,,,,,,,
1980,1,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1984,1,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1986,2,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1987,4,5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1988,2,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1989,9,10,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1990,20,34,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1991,8,11,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1992,35,45,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
